# 1. Imports

In [1]:
#necessary libraries
from pathlib import Path
import pandas as pd  
import numpy as np  
import warnings  
from unidecode import unidecode
import re  
import matplotlib.pyplot as plt
import seaborn as sns
import time
from rapidfuzz import process, fuzz
import random
import scipy.stats as st
import math
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
 
#commands for better output readability 
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
warnings.filterwarnings("ignore", category=UserWarning, module='pandas')  
pd.set_option('display.max_colwidth', None)

In [3]:
#paths
project_dir=Path.cwd().parent.parent
processed_dir=project_dir/'data'/'processed'

final_file=processed_dir/'matched_grouped_final.pkl'
labeled_file=processed_dir/'matched_grouped_labeled_sample.pkl'

df=pd.read_pickle(final_file)  
sample=pd.read_pickle(labeled_file)  

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608 entries, 0 to 3607
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UK ID                   3608 non-null   int64  
 1   UK Sanction Programme   3608 non-null   object 
 2   UK Name                 3608 non-null   object 
 3   UK Letters              3608 non-null   object 
 4   Candidate EU IDs        3608 non-null   object 
 5   Candidate Count         3608 non-null   int64  
 6   Name Overlap            3532 non-null   object 
 7   EU Name Match           3532 non-null   object 
 8   EU ID                   3532 non-null   float64
 9   Multi Score             3608 non-null   int64  
 10  Coverage Ratio          3608 non-null   float64
 11  Length Adjusted Scores  3532 non-null   object 
 12  Weighted Score          3608 non-null   float64
 13  Raw Scores              3532 non-null   object 
 14  Label                   3608 non-null   

In [5]:
sample.head()

,UK ID,UK Sanction Programme,UK Name,UK Letters,Candidate EU IDs,Candidate Count,Name Overlap,EU Name Match,EU ID,Multi Score,Coverage Ratio,Length Adjusted Scores,Weighted Score,Raw Scores,Label,True Label
0,6894,ISIL (Da'esh) and Al-Qaida,"{Fihiruddin, Jibril, Iqbal, Abdurrahman, Abdul, Muqti, A, Rahman, Fikiruddin, Mohamad, Abu}","{J, R, F, I, A, M}","[630, 643, 1004, 3140, 4686, 5240, 5262, 5271, 5623, 6133, 6211, 6478, 6494, 6830, 6974, 7250, 113355, 115714, 117974, 123615, 125562, 126101, 127538, 129864, 130225, 133935, 134828, 135060, 136530, 136975, 138176, 145803, 146560, 147032, 150914, 159126, 162479, 162975, 165652, 166799, 167049, 167477, 171171, 172374, 172394]",45,"[Fikiruddin, Rahman, Mohamad, A, Abdul, Jibril, Iqbal, Muqti, Abdurrahman]","{Fihiruddin, Jibril, Iqbal, Abdurrahman, Abdul, Muqti, A, Rahman, Fikiruddin, Mohamad, Abu}",1004.0,98,1.0,"[100, 100.0, 100.0, 87.0, 95.0, 100.0, 95.0, 95.0, 100]",97.666667,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match,match
1,6905,Afghanistan,"{Zakir, Bari, Haji, Mullah, Akhund, Abdul, Sahib}","{B, S, Z, H, A, M}","[25, 46, 54, 103, 514, 515, 528, 553, 556, 593, 595, 599, 600, 603, 604, 661, 727, 750, 927, 928, 937, 965, 966, 1016, 1065, 1069, 1085, 1102, 1894, 1896, 2193, 2208, 3084, 3144, 3225, 3341, 3361, 3663, 3808, 4142, 5294, 5295, 5416, 5417, 5529, 5616, 5619, 5625, 6044, 6047, 6049, 6052, 6053, 6054, 6073, 6074, 6078, 6079, 6084, 6085, 6090, 6116, 6130, 6213, 6223, 6238, 6309, 6484, 6494, 6607, 6608, 6616, 6695, 6898, 6947, 6982, 7065, 7069, 7074, 7166, 7199, 7223, 7231, 7443, 7472, 7474, 7475, 7477, 7483, 7484, 7496, 7597, 106138, 106144, 106385, 106389, 106534, 106538, 106584, 106654, ...]",196,"[Mullah, Sahib, Zakir, Bari, Abdul, Akhund, Haji]","{Zakir, Bari, Haji, Mullah, Akhund, Abdul, Sahib}",556.0,97,1.0,"[100.0, 95.0, 95.0, 95.0, 95.0, 100.0, 95.0]",97.321429,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match,match
2,6912,Afghanistan,"{Khadem, Abdul, Rauf, Aliza}","{R, K, A}","[143, 146, 513, 659, 676, 719, 724, 786, 849, 1096, 2921, 3062, 3140, 3342, 3793, 5266, 5925, 6212, 6215, 6216, 6240, 6582, 6584, 6650, 6945, 6974, 6989, 6993, 7085, 7221, 7229, 7237, 7250, 7291, 7361, 7368, 7405, 7435, 7455, 7456, 7504, 7513, 7519, 7523, 7567, 107818, 108640, 109244, 109980, 110174, 110425, 113326, 117485, 117974, 119548, 119656, 123615, 125538, 125637, 126338, 126507, 126698, 127526, 127528, 127726, 128270, 128279, 129815, 129836, 129884, 130225, 130355, 132522, 133907, 133959, 134035, 134083, 134107, 134171, 134175, 134340, 134484, 134488, 134532, 134556, 134684, 134860, 134872, 134876, 134880, 134884, 134892, 134900, 134920, 134924, 134928, 134932, 134936, 134964, 134976, ...]",248,"[Abdul, Rauf, Khadem, Aliza]","{Khadem, Mullah, Abdul, Aliza, Rauf}",719.0,97,1.0,"[95.0, 95.0, 100.0, 95.0]",97.187500,"[100.0, 100.0, 100.0, 100.0]",match,match
3,6932,ISIL (Da'esh) and Al-Qaida,"{Mahmoud, Umr, Filistini, Al, Takfiri, Umar, Uthman, Mohammed, Othman, Samman, Omar, Abu, Qatada, Ismail}","{S, O, Q, T, F, U, I, A, M}","[23, 58, 157, 550, 836, 3081, 3082, 3083, 3225, 3962, 5609, 6515, 7025, 7478, 7484, 7497, 115714, 117251, 139991, 152885]",20,"[Othman, Omar, Umr, Qatada, Ismail, Al, Filistini, Mahmoud, Samman, Mohammed, Takfiri, Abu]","{Mahmoud, Umr, Filistini, Al, Umar, Uthman, Ismail, Mohammed, Othman, Samman, Omar, Abu, Qatada, Takfiri}",836.0,97,1.0,"[100.0, 95.0, 87.0, 100.0, 100.0, 87.0, 100, 100.0, 100.0, 100.0, 100.0, 87.0]",97.250000,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match,match
4,7024,ISIL (Da'esh) and Al-Qaida,"{Belgacem, Al, Aouadi, Belkacem, Mohamed, Abdallah, Fathi, Hannachi, Ben}","{B, F, H, A, M}","[25, 54, 69, 89, 103, 139, 175, 191, 528, 544, 556, 603, 651, 653, 661, 669, 702, 706, 923, 927, 928, 1016, 1065, 1085, 1894, 2700, 3000, 3060, 3144, 3181, 4140, 5416, 5417, 5500, 5619, 5623, 5625, 6047, 6049, 6052, 6074, 6078, 6079, 6080, 6082, 6096, 6118, 6130, 6133, 6213, 6

In [6]:
def feature_engineering(df):


    def get_count(token_set_or_list):

        if isinstance(token_set_or_list, (set, list)):
            return len(token_set_or_list)
            
        return 0

    
    def get_average(list_scores):

        if isinstance(list_scores, list) and len(list_scores) > 0:
            return round(sum(list_scores)/len(list_scores), 1)
            
        return 0



    df['uk_letters_count']=df['UK Letters'].apply(get_count)
    df['candidate_count']=df['Candidate Count']
    df['uk_name_count']=df['UK Name'].apply(get_count)
    df['overlap_name_count']=df['UK Name'].apply(get_count)
    df['eu_name_match_count']=df['EU Name Match'].apply(get_count)
    df['multi_score']=df['Multi Score']
    df['coverage_ratio']=df['Coverage Ratio']
    df['length_adj_avg_score']=df['Length Adjusted Scores'].apply(get_average)
    df['avg_raw_score']=df['Raw Scores'].apply(get_average)
    
    
    columns_to_drop=['UK Sanction Programme',
                     'UK Name',
                     'UK Letters',
                     'Candidate EU IDs',
                     'Name Overlap',
                     'EU Name Match',
                     'Multi Score',
                     'Coverage Ratio',
                     'Length Adjusted Scores',
                     'Weighted Score',
                     'Raw Scores',
                     'Label']
    
    df=df.drop(columns=columns_to_drop)
    
    return df

In [7]:
feature_engineering(df)
feature_engineering(sample)

,UK ID,Candidate Count,EU ID,True Label,uk_letters_count,candidate_count,uk_name_count,overlap_name_count,eu_name_match_count,multi_score,coverage_ratio,length_adj_avg_score,avg_raw_score
0,6894,45,1004.0,match,6,45,11,11,11,98,1.000000,96.9,100.0
1,6905,196,556.0,match,6,196,7,7,7,97,1.000000,96.4,100.0
2,6912,248,719.0,match,3,248,4,4,5,97,1.000000,96.2,100.0
3,6932,20,836.0,match,9,20,14,14,14,97,1.000000,96.3,100.0
4,7024,128,927.0,match,5,128,9,9,9,97,1.000000,96.1,100.0
5,7099,99,557.0,match,2,99,3,3,4,96,1.000000,94.0,100.0
6,7278,215,649.0,not match,1,215,1,1,1,50,1.000000,100.0,100.0
7,7296,159,651.0,match,4,159,7,7,7,98,1.000000,96.7,100.0
8,7380,198,702.0,match,4,198,7,7,7,98,1.000000,97.0,100.0
9,7442,240,758.0,match,3,240,3,3,3,98,1.000000,96.7,100.0
